# 0 YouTube Crawling

#### 유튜브 내 검색 결과의 콘텐츠 정보 자동화를 통하여 '제페토커스텀' 과 관련한 영상과 조회수를 알아본다

# 1 라이브러리 로드 

In [1]:
# 최신 크롬 드라이버 사용하도록 세팅: 현재 OS에 설치된 크롬 브라우저 버전에 맞게 cache에 드라이버 설치
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# 최신 크롬 드라이버 사용하도록 세팅: 현재 OS에 설치된 크롬 브라우저 버전에 맞게 cache에 드라이버 설치
from selenium.webdriver.chrome.service import Service
service = Service(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\LG\.wdm\drivers\chromedriver\win32\103.0.5060.134\chromedriver.exe] found in cache


In [12]:
#판다스를 통하여 수집한 데이터들을 데이터프레임으로 만들어 줄 예정이다
import time
import random
import os
import pandas as pd

In [4]:
class YoutubeCrawler:
    def __init__(self):
        self._opts = Options()
        # self._opts.add_argument("headless")
        self._opts.add_argument("window-size=1920x1080")
        self._opts.add_argument("disable-gpu")
        self._opts.add_argument("--no-sandbox")
        self._opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
        #if conf.binary_location is not None:
        #    self._opts.binary_location = conf.binary_location
        self._driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=self._opts)
        self._driver.implicitly_wait(3)
        time.sleep(3)
    
    @property
    def driver(self):
        return self._driver

# 2 데이터 스크랩

In [5]:
search_keywords = []
search_keywords.append('제페토 커스텀')
search_keywords.append('제페토')

In [13]:
crawler = YoutubeCrawler()

df = pd.DataFrame(columns = columns)

for search_keyword in search_keywords:
    url = 'https://www.youtube.com/results?search_query='
    param = '+'.join(search_keyword.split(" "))
    url = url + param

    crawler.driver.implicitly_wait(3)
    crawler.driver.get(url)

    ######
    time.sleep(5)

    crawler.driver.implicitly_wait(1)
    scroll_count = 500
    for _ in range(scroll_count):
        crawler.driver.execute_script("window.scrollTo(0, 999999);")
        time.sleep(0.1)

    columns = ['title', 'viewcount']
    contents_el = crawler.driver.find_elements(By.CSS_SELECTOR, "#contents")
    crawler.driver.implicitly_wait(0)
    for content_el in contents_el:
        rows_el = content_el.find_elements(By.CSS_SELECTOR, "ytd-video-renderer")
        for row_el in rows_el:
            title_el = row_el.find_elements(By.CSS_SELECTOR, "#video-title")
            viewcount_el = row_el.find_elements(By.CSS_SELECTOR, "#metadata-line > span:nth-child(1)")
            if len(title_el) > 0 and len(viewcount_el) > 0:
                row = pandas.DataFrame([title_el[0].text, viewcount_el[0].text]).T
                row.columns = columns
                if row['title'][0] not in list(df['title']):
                    df = pandas.concat([df, row])

df.to_csv("all.csv")

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.134/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\LG\.wdm\drivers\chromedriver\win32\103.0.5060.134]


NameError: name 'columns' is not defined

# 4 데이터 추출

# 5 데이터프레임 저장